In [38]:
!nvidia-smi

Mon Mar 29 11:41:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.06    Driver Version: 450.51.06    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    35W / 250W |    367MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 1-Import libraries and Load files

In [20]:
# Import libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('max_colwidth', 500)

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import os, sys, gc, warnings, random
warnings.filterwarnings('ignore')

from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from sklearn.metrics import auc, classification_report, roc_auc_score
from sklearn.model_selection import StratifiedKFold

import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier ,Pool

In [21]:
!pip install catboost==0.22 --quiet

In [22]:
# File PATH
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/umoja2021/SampleSubmission.csv
/kaggle/input/umoja2021/Train.csv
/kaggle/input/umoja2021/VariableDefinitions.csv
/kaggle/input/umoja2021/Test.csv


In [23]:
# Load files
train = pd.read_csv('/kaggle/input/umoja2021/Train.csv')
test = pd.read_csv('/kaggle/input/umoja2021/Test.csv')
samplesubmission = pd.read_csv('/kaggle/input/umoja2021/SampleSubmission.csv')
variable_definations = pd.read_csv('/kaggle/input/umoja2021/VariableDefinitions.csv')

# 2-Preprocessing
 

In [24]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
SEED = 42    
seed_everything(SEED) 

In [25]:
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'country'.
train['country_code']= label_encoder.fit_transform(train['country'])

test['country_code']= label_encoder.transform(test['country'])

In [26]:
# label_encoder object knows how to understand word labels.
label_encoder2 = preprocessing.LabelEncoder()
  
# Encode labels in column 'region'.
train['region_code']= label_encoder2.fit_transform(train['region'].astype(str))

test['region_code']= label_encoder2.transform(test['region'].astype(str))

# 3-Modeling and Validation


In [27]:
remove_features = ['ID', 'country', 'region','target']
features_columns = [col for col in train.columns if col not in remove_features]
categ_features = ['country_code','region_code']
X,y   = train[features_columns], train['target']

# 3.1-Catboost

In [28]:
score = 0
test['target'] = 0
skf = StratifiedKFold(n_splits=5,shuffle=True, random_state=SEED)
oof_cat = np.zeros((train.shape[0],))
cat_preds= []
for fold_, (trn_idx, val_idx) in enumerate(skf.split(X, train.country)):
    print('Fold:',fold_+1)
        
    # Creating lgb train/valid data
    X_train, y_train = X.iloc[trn_idx,:], y[trn_idx] 
    X_test, y_test = X.iloc[val_idx,:], y[val_idx] 
       
    estimator = CatBoostClassifier(learning_rate=0.05,
                                task_type="GPU",
                                devices='0:1',
                                iterations=10000,eval_metric='AUC',
                              use_best_model =True,
                              verbose=100,
                             random_seed= 0)
    estimator.fit(Pool(X_train,y_train,cat_features = categ_features),
                  eval_set = Pool(X_test,y_test,cat_features = categ_features),early_stopping_rounds=200)
    
    y_pred_val = estimator.predict_proba(X_test)[:,1]
    oof_cat[val_idx] = y_pred_val
    score = score + roc_auc_score(y_test, y_pred_val)
    y_pred_test = estimator.predict_proba(test[features_columns])[:,1]
    cat_preds.append(y_pred_test)
print('OOF score :',roc_auc_score(y, oof_cat)) 

Fold: 1
0:	learn: 0.7462481	test: 0.7486294	best: 0.7486294 (0)	total: 89ms	remaining: 14m 50s
100:	learn: 0.7948139	test: 0.7951516	best: 0.7951516 (100)	total: 3.17s	remaining: 5m 10s
200:	learn: 0.8018712	test: 0.7992482	best: 0.7992482 (200)	total: 5.82s	remaining: 4m 43s
300:	learn: 0.8065880	test: 0.8011628	best: 0.8011628 (300)	total: 8.69s	remaining: 4m 39s
400:	learn: 0.8104563	test: 0.8023691	best: 0.8023691 (400)	total: 11.4s	remaining: 4m 32s
500:	learn: 0.8136608	test: 0.8030019	best: 0.8030076 (492)	total: 14s	remaining: 4m 26s
600:	learn: 0.8164281	test: 0.8034564	best: 0.8034607 (598)	total: 16.7s	remaining: 4m 21s
700:	learn: 0.8189620	test: 0.8037325	best: 0.8037422 (694)	total: 19.6s	remaining: 4m 20s
800:	learn: 0.8213338	test: 0.8040116	best: 0.8040350 (794)	total: 22.3s	remaining: 4m 16s
900:	learn: 0.8237124	test: 0.8042645	best: 0.8042771 (890)	total: 25s	remaining: 4m 12s
1000:	learn: 0.8257954	test: 0.8044313	best: 0.8044551 (996)	total: 27.7s	remaining: 4m 8s

1500:	learn: 0.8356773	test: 0.8051198	best: 0.8051291 (1498)	total: 42.4s	remaining: 4m
1600:	learn: 0.8373988	test: 0.8051282	best: 0.8051570 (1535)	total: 45.1s	remaining: 3m 56s
1700:	learn: 0.8392369	test: 0.8049975	best: 0.8051570 (1535)	total: 47.8s	remaining: 3m 53s
bestTest = 0.8051570356
bestIteration = 1535
Shrink model to first 1536 iterations.
OOF score : 0.8036701848165237


In [29]:
catboost_preds = np.mean(cat_preds,axis=0)

# 3.2-LGBM

In [30]:
########################### Models params and Features
###########################################################

lgb_params = {'boosting_type': 'gbdt','objective': 'binary','metric': 'auc',
              'n_estimators': 500,'early_stopping_rounds': 100,'sub_sample' : 0.7,
              'colsample_bytree' : 0.6,'seed': SEED,'silent':False
            }

In [31]:
score = 0
test['target'] = 0
skf = StratifiedKFold(n_splits=5,shuffle=True, random_state=SEED)
oof_lgb = np.zeros((train.shape[0],))
lgb_preds = []
for fold_, (trn_idx, val_idx) in enumerate(skf.split(X, train.country)):
    print('Fold:',fold_+1)
        
    # Creating lgb train/valid data
    tr_x, tr_y = X.iloc[trn_idx,:], y[trn_idx] 
    vl_x, vl_y = X.iloc[val_idx,:], y[val_idx] 
        
    train_data = lgb.Dataset(tr_x, label=tr_y,categorical_feature=categ_features)
    valid_data = lgb.Dataset(vl_x, label=vl_y,categorical_feature=categ_features)
    # Train Model
    
    estimator = lgb.train(
                          lgb_params,
                          train_data,
                          valid_sets = [train_data,valid_data],
                          verbose_eval = 100,
                        )

    y_pred_val = estimator.predict(vl_x,num_iteration=estimator.best_iteration)
    oof_lgb[val_idx] = y_pred_val
    score = score + roc_auc_score(vl_y, y_pred_val)
    y_pred_test = estimator.predict(test[features_columns],num_iteration=estimator.best_iteration)
    lgb_preds.append(y_pred_test)

print('OOF score :',roc_auc_score(y, oof_lgb))

Fold: 1
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Unknown parameter: sub_sample
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Unknown parameter: sub_sample
[LightGBM] [Info] Number of positive: 48828, number of negative: 35065
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.014620 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 549
[LightGBM] [Info] Number of data points in the train set: 83893, number of used features: 34
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Unknown parameter: sub_sample
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.582027 -> initscore=0.331100
[LightGBM] [Info] Start training from score 0.331100
Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.825801	valid_1's auc: 0.803917
[200]	training's auc: 0

In [32]:
lightgbm_preds = np.mean(lgb_preds,axis=0)

# 3.3-Xgboost

In [33]:
########################### Models params and Features
###########################################################

param = {'objective': 'binary:logistic','eval_metric': 'auc','subsample': 0.8,'colsample_bytree' : 0.9,'learning_rate':0.02,'random_state':SEED, 'seed':SEED,'gpu_id': 0, 'booster': 'gbtree','tree_method': 'gpu_hist', 'grow_policy':'lossguide'}

In [34]:
score = 0
test['target'] = 0
skf = StratifiedKFold(n_splits=5,shuffle=True, random_state=SEED)
oof_xgb = np.zeros((train.shape[0],))
xgb_preds = []
for fold_, (trn_idx, val_idx) in enumerate(skf.split(X, train.country)):
    print('Fold:',fold_+1)
        
    # Creating lgb train/valid data
    tr_x, tr_y = X.iloc[trn_idx,:], y[trn_idx] 
    vl_x, vl_y = X.iloc[val_idx,:], y[val_idx] 
        
    train_data = xgb.DMatrix(tr_x, label=tr_y)
    valid_data = xgb.DMatrix(vl_x, label=vl_y)
    ###
    evals=[(train_data, 'train'), (valid_data, 'valid')]
    estimator_clf = xgb.train(param,train_data,evals=evals,num_boost_round=10000, 
        
        verbose_eval = 100,early_stopping_rounds=100)
    

    y_pred_val = estimator_clf.predict(xgb.DMatrix(vl_x))
    oof_xgb[val_idx] = y_pred_val
    score = score + roc_auc_score(vl_y, y_pred_val)
    y_pred_test = estimator_clf.predict(xgb.DMatrix(test[features_columns]))
    xgb_preds.append(y_pred_test)

print('OOF score :',roc_auc_score(y, oof_xgb))

Fold: 1
[0]	train-auc:0.74145	valid-auc:0.73889
[100]	train-auc:0.78064	valid-auc:0.77392
[200]	train-auc:0.79850	valid-auc:0.78748
[300]	train-auc:0.80930	valid-auc:0.79452
[400]	train-auc:0.81595	valid-auc:0.79801
[500]	train-auc:0.82084	valid-auc:0.80011
[600]	train-auc:0.82438	valid-auc:0.80110
[700]	train-auc:0.82779	valid-auc:0.80196
[800]	train-auc:0.83074	valid-auc:0.80244
[900]	train-auc:0.83364	valid-auc:0.80274
[1000]	train-auc:0.83617	valid-auc:0.80306
[1100]	train-auc:0.83855	valid-auc:0.80327
[1200]	train-auc:0.84102	valid-auc:0.80334
[1300]	train-auc:0.84342	valid-auc:0.80347
[1397]	train-auc:0.84556	valid-auc:0.80342
Fold: 2
[0]	train-auc:0.74206	valid-auc:0.73457
[100]	train-auc:0.78121	valid-auc:0.76926
[200]	train-auc:0.79868	valid-auc:0.78281
[300]	train-auc:0.80974	valid-auc:0.79092
[400]	train-auc:0.81657	valid-auc:0.79498
[500]	train-auc:0.82125	valid-auc:0.79691
[600]	train-auc:0.82531	valid-auc:0.79837
[700]	train-auc:0.82855	valid-auc:0.79916
[800]	train-auc:0

In [35]:
xgboost_preds = np.mean(xgb_preds,axis=0)

# 4-Let's try Local Ensemble

In [36]:
#Validation Ensemble
blend = oof_cat*0.5+ oof_lgb*0.4 + oof_xgb*0.1
roc_auc_score(y,blend ) 

0.805004547455695

# 5-Submission

In [37]:
test['target'] =catboost_preds*0.5+  lightgbm_preds*0.4 +xgboost_preds*0.1
submission = test[['ID', 'target']]
submission.to_csv('UmojaHack-Challenge#3-Top1-Solution.csv',index = False)